In [6]:
# Python
# importing libraries
!pip install pulp
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize
import time

# Create the objective function coefficients
M = 10^3
c = np.array([8, 6, 3, 2, 4, 9, 0, 0, 0, 0, 0, M, M, M])

# Create the constraint matrix
A = np.array([[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
              [1, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0],
              [0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 1, 0],
              [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 1]])

# Create the right-hand side of the constraints
b = np.array([70, 40, 40, 35, 25])

# Create the problem
problem = LpProblem("Column Generation", LpMinimize)

# Create the variables
x = [LpVariable(f"x{i+1}", lowBound=0) for i in range(14)]

# Set the objective function
problem += np.dot(c, x), "Objective Function"

# Add the constraints
for i in range(len(b)):
    problem += np.dot(A[i], x) == b[i]

# Measure the runtime
start_time = time.time()

# Solve the problem
problem.solve()

end_time = time.time()
runtime = end_time - start_time

# Print the results
for v in problem.variables():
    print(f"{v.name}: {v.value()}")

print(f"Optimal value: {problem.objective.value()}")
print(f"Runtime: {runtime:.6f} seconds")

x1: 0.0
x10: 0.0
x11: 0.0
x12: 0.0
x13: 0.0
x14: 0.0
x2: 35.0
x3: 25.0
x4: 40.0
x5: 0.0
x6: 0.0
x7: 10.0
x8: 0.0
x9: 0.0
Optimal value: 365.0
Runtime: 0.013992 seconds


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [3]:
# Python
# Original Source: https://github.com/demirayonur/Column-Generation.git
# importing libraries
!pip install gurobipy
from gurobipy import GRB
import gurobipy as gp
import numpy as np
import math

# Define a class to represent the instance of the problem
class Instance(object):

    def __init__(self, order_lengths, order_demands):
        # Initialize instance attributes
        self.n = len(order_lengths)  # Number of orders
        self.order_lens = order_lengths  # Length of each order
        self.demands = order_demands  # Demand for each order
        self.m = np.sum(self.demands)  # Total demand
        self.roll_len = 15  # Length of the roll

    def summarize(self):
        # Summarize the problem instance
        print("Problem instance with ", self.n, " orders and ", self.m, "rolls")
        print("-"*47)
        print("\nOrders:\n")
        for i, order_len in enumerate(self.order_lens):
            print("\tOrder ", i, ": length= ", order_len, " demand=", self.demands[i])
        print("\nRoll Length: ", self.roll_len)

# Function to generate initial cutting patterns
def generate_initial_patterns(ins_:Instance):
    patterns = []

    # Define initial cutting patterns
    pattern1 = [3, 0, 0]
    pattern2 = [0, 2, 0]
    pattern3 = [0, 0, 2]
    pattern4 = [2, 0, 1]
    pattern5 = [2, 1, 0]
    pattern6 = [0, 1, 1]

    # Add patterns to the list
    patterns.append(pattern1)
    patterns.append(pattern2)
    patterns.append(pattern3)
    patterns.append(pattern4)
    patterns.append(pattern5)
    patterns.append(pattern6)

    # Print the patterns and their values
    for i, pattern in enumerate(patterns):
        print(f"Pattern {i}: {pattern}")
        print(f"Value: {sum(pattern)}")
        print("-------------------")

    return patterns

# Function to define the master problem (linear programming formulation)
def define_master_problem(ins_:Instance, patterns):
    n_pattern = len(patterns)
    pattern_range = range(n_pattern)
    order_range = range(ins_.n)
    patterns = np.array(patterns, dtype=int)
    master_problem = gp.Model("master problem")

    # Decision variables (lambda variables)
    lambda_ = master_problem.addVars(pattern_range,
                                     vtype=GRB.CONTINUOUS,
                                     obj=np.ones(n_pattern),
                                     name="lambda")

    # Direction of optimization (minimization)
    master_problem.modelSense = GRB.MINIMIZE

    # Demand satisfaction constraint
    for i in order_range:
        master_problem.addConstr(sum(patterns[p,i]*lambda_[p] for p in pattern_range) == ins_.demands[i],
                                 "Demand[%d]" %i)

    # Solve the master problem
    return master_problem

# Function to define the subproblem (knapsack problem)
def define_subproblem(ins_:Instance, duals):
    order_range = range(ins_.n)
    subproblem = gp.Model("subproblem")

    # Decision variables (x variables)
    x = subproblem.addVars(order_range,
                           vtype=GRB.INTEGER,
                           obj=duals,
                           name="x")

    # Direction of optimization (maximization)
    subproblem.modelSense = GRB.MAXIMIZE

    # Length constraint (knapsack constraint)
    subproblem.addConstr(sum(ins_.order_lens[i] * x[i] for i in order_range) <= ins_.roll_len)

    return subproblem

# Function to print the solution of the master problem
def print_solution(master, patterns):
    use = [math.ceil(i.x) for i in master.getVars()]
    for i, p in enumerate(patterns):
        if use[i]>0:
            print('Pattern ', i, ': how often we should cut: ', use[i])
            print('----------------------')
            for j,order in enumerate(p):
                if order >0:
                    print('order ', j, ' how much: ', order)
            print()

# Column generation algorithm
def column_generation(ins_:Instance):
    patterns = generate_initial_patterns(ins_)
    objVal_history = []

    while True:
        # Define and solve the master problem
        master_problem = define_master_problem(ins_, patterns)
        master_problem.optimize()
        objVal_history.append(master_problem.objVal)

        # Obtain dual variables from the master problem
        dual_variables = np.array([constraint.pi for constraint in master_problem.getConstrs()])

        # Define and solve the subproblem
        subproblem = define_subproblem(ins_, dual_variables)
        subproblem.optimize()

        # Check termination condition
        if subproblem.objVal < 1 + 1e-2:
            break
        patterns.append([i.x for i in subproblem.getVars()])

    # Print the solution
    print_solution(master_problem, patterns)
    print('Total number of rolls used: ', int(np.array([math.ceil(i.x) for i in master_problem.getVars()]).sum()))
    return objVal_history

# New instance details
order_lengths = [4, 6, 7]
order_demands = [80, 50, 100]

# Create an instance object
instance = Instance(order_lengths, order_demands)
instance.summarize()

# Run the column generation algorithm and store the objective function values
history = column_generation(instance)


Problem instance with  3  orders and  230 rolls
-----------------------------------------------

Orders:

	Order  0 : length=  4  demand= 80
	Order  1 : length=  6  demand= 50
	Order  2 : length=  7  demand= 100

Roll Length:  15
Pattern 0: [3, 0, 0]
Value: 3
-------------------
Pattern 1: [0, 2, 0]
Value: 2
-------------------
Pattern 2: [0, 0, 2]
Value: 2
-------------------
Pattern 3: [2, 0, 1]
Value: 3
-------------------
Pattern 4: [2, 1, 0]
Value: 3
-------------------
Pattern 5: [0, 1, 1]
Value: 2
-------------------
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 6 columns and 9 nonzeros
Model fingerprint: 0xcf8f8c52
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+0